In [1]:
import numpy as np
np.random.seed(2020)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import statistics
import random
import time
import tensorflow.compat.v1 as tf
import keras

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras import backend as K

from sklearn.metrics import log_loss
from imageio import imread

from PIL import Image

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
K.tensorflow_backend._get_available_gpus()


#globals for rows and cols since we will always be doing images of the same size and they will always be 3 for RGB or 1 for Gray
rows = 60
cols = 80
RGB = 3


Using TensorFlow backend.


In [2]:
def create_model():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(32, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model2():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model3():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    #model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    #model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    #model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model4():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model5():
    model = Sequential()

    ## CNN 1
    model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(RGB, rows, cols)))
    model.add(BatchNormalization())
    model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 2
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.3))

    ## CNN 3
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization(axis = 3))
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))

    ## Output
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10,activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_model6():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def create_model7():
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(RGB, rows, cols), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='adam', loss='kullback_leibler_divergence', metrics=['accuracy'])
    return model

def getTestImg(fPath):
    img = Image.open(fPath)
    return np.asarray(img)/255

In [3]:
    model = Sequential()
    model.add(Convolution2D(32, (3, 3), input_shape=(rows, cols, RGB), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Convolution2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

    model.add(Flatten())
    model.add(Dense(10))
    model.add(Activation('softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 80, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 80, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 60, 80, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 80, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 40, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 40, 64)        9280      
_________________________________________________________________
batch_normalization_2 (Batch (None, 60, 40, 64)       

In [4]:
train_datagen = ImageDataGenerator(rescale = 1.0/255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   validation_split = 0.2)

training_set = train_datagen.flow_from_directory('train', 
                                                 target_size = (rows, cols), 
                                                 batch_size = 32,
                                                 subset = 'training')

validation_set = train_datagen.flow_from_directory('train', 
                                                   target_size = (rows, cols), 
                                                   batch_size = 32,
                                                   subset = 'validation')

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [6]:
model.fit_generator(training_set,
                         steps_per_epoch = 17943/32,
                         epochs = 20,
                         validation_data = validation_set,
                         validation_steps = 4481/32)



Epoch 1/20
561/560 [==============================] - 24s 43ms/step - loss: 0.1526 - accuracy: 0.9542 - val_loss: 0.0873 - val_accuracy: 0.6704
Epoch 2/20
561/560 [==============================] - 23s 41ms/step - loss: 0.1461 - accuracy: 0.9564 - val_loss: 0.3065 - val_accuracy: 0.6717
Epoch 3/20
561/560 [==============================] - 24s 43ms/step - loss: 0.1291 - accuracy: 0.9598 - val_loss: 1.1197 - val_accuracy: 0.6699
Epoch 4/20
561/560 [==============================] - 23s 41ms/step - loss: 0.1306 - accuracy: 0.9625 - val_loss: 0.9255 - val_accuracy: 0.6983
Epoch 5/20
561/560 [==============================] - 23s 41ms/step - loss: 0.1225 - accuracy: 0.9636 - val_loss: 1.7691 - val_accuracy: 0.6894
Epoch 6/20
561/560 [==============================] - 23s 41ms/step - loss: 0.1191 - accuracy: 0.9659 - val_loss: 0.0020 - val_accuracy: 0.7268
Epoch 7/20
561/560 [==============================] - 23s 41ms/step - loss: 0.1095 - accuracy: 0.9677 - val_loss: 2.2509 - val_accuracy:

In [7]:
test_file = pd.read_csv('sample_submission.csv')
test_file.head(5)
for i, file in enumerate(test_file['img']):
    image = getTestImg('test/' + file)
    image = np.reshape(image, (1, rows, cols, RGB))
    result = model.predict(image)
    test_file.iloc[i, 1:] = result[0]
    
test_file.to_csv('submission.csv', index = False)